In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup

In [77]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://www.premierleague.com/managers?se=-1&cl=-1"
driver.get(url)

In [78]:
time.sleep(5)

In [79]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [80]:
table = soup.find('div', {'class':'table'})
rows = table.find_all('tr')

In [81]:
manager_name = []
team_name = []

In [82]:
for row in rows[1:]:
    cols = row.find_all('td')
    if len(cols) > 0:
        manager_name.append(cols[0].text.strip())
        
        team = cols[1].find('a').text.strip().split()[0]
        team_name.append(team)

In [83]:
df = pd.DataFrame({
    'Manager_Name': manager_name,
    'Team_Name': team_name
})

In [84]:
df2 = pd.read_csv('League Power Ranking.csv')

In [85]:
def correct_team_names(row):
    manager_name = row['Manager_Name']
    team_name = row['Team_Name']
    
    if 'Manchester' in team_name:
        if manager_name == 'Erik ten Hag' or manager_name in [
            'Ralf Rangnick', 'Michael Carrick', 'Louis van Gaal', 
            'Ryan Giggs', 'Ole Gunnar Solskjær', 'Alex Ferguson']:
            return 'Manchester United'
        else:
            return 'Manchester City'
    
    if 'Sheffield' in team_name:
        if manager_name in ['Peter Shreeves', 'Danny Wilson', 'Trevor Francis']:
            return 'Sheffield Wednesday'
        else:
            return 'Sheffield United'
    
    return team_name

# %% Apply the correction function for team names
df['Team_Name'] = df.apply(correct_team_names, axis=1)

# %% Function to match team names between df and df2
def match_and_replace_team_name(df_team_name, df2):
    # Try exact match first
    for idx, row in df2.iterrows():
        if df_team_name == row['Team_Name']:
            return row['Team_Name'], row['Team_ID']
    
    # If no exact match, try matching the first word
    df_team_first_word = df_team_name.split()[0]
    for idx, row in df2.iterrows():
        team_first_word = row['Team_Name'].split()[0]
        if df_team_first_word == team_first_word:
            return row['Team_Name'], row['Team_ID']
    
    # If no match, return the original name with None as Team_ID
    return df_team_name, None

# %% Apply the matching function to correct team names in df and get Team_ID
df['Team_Name'], df['Team_ID'] = zip(*df['Team_Name'].apply(lambda x: match_and_replace_team_name(x, df2)))

# %% Function to generate Manager_ID using initials and total letter count
def generate_manager_id(manager_name, team_id):
    first_initial = manager_name[0]  # First letter of first name
    last_initial = manager_name.split(' ')[1][0]  # First letter of last name
    total_letter_count = len(manager_name.replace(' ', ''))  # Total letter count without spaces
    
    # Format the Manager_ID
    manager_id = f"{first_initial}{last_initial}{team_id}{str(total_letter_count).zfill(2)}"
    
    return manager_id

# %% Apply the Manager_ID generation function
df['Manager_ID'] = df.apply(lambda row: generate_manager_id(row['Manager_Name'], row['Team_ID']), axis=1)

# %% Reorganize columns and display the final DataFrame
df = df[['Manager_ID', 'Manager_Name', 'Team_Name', 'Team_ID']]

df.head()

,Manager_ID,Manager_Name,Team_Name,Team_ID
0,MAARS11,Mikel Arteta,Arsenal,ARS
1,FLARS16,Fredrik Ljungberg,Arsenal,ARS
2,AWARS12,Arsène Wenger,Arsenal,ARS
3,BRARS10,Bruce Rioch,Arsenal,ARS
4,SHARS14,Stewart Houston,Arsenal,ARS


In [86]:
df.head(60)

,Manager_ID,Manager_Name,Team_Name,Team_ID
0,MAARS11,Mikel Arteta,Arsenal,ARS
1,FLARS16,Fredrik Ljungberg,Arsenal,ARS
2,AWARS12,Arsène Wenger,Arsenal,ARS
3,BRARS10,Bruce Rioch,Arsenal,ARS
4,SHARS14,Stewart Houston,Arsenal,ARS
5,ADAVI10,Aaron Danks,Aston Villa,AVI
6,SGAVI13,Steven Gerrard,Aston Villa,AVI
7,UEAVI09,Unai Emery,Aston Villa,AVI
8,RGAVI09,Rémi Garde,Aston Villa,AVI
9,TSAVI11,Tim Sherwood,Aston Villa,AVI


In [87]:
df.tail()

,Manager_ID,Manager_Name,Team_Name,Team_ID
273,GOWWA10,Gary O'Neil,Wolverhampton Wanderers,WWA
274,BMWWA28,Bruno Miguel Silva do Nascimento,Wolverhampton Wanderers,WWA
275,TCWWA11,Terry Connor,Wolverhampton Wanderers,WWA
276,MMWWA12,Mick McCarthy,Wolverhampton Wanderers,WWA
277,DJWWA09,Dave Jones,Wolverhampton Wanderers,WWA


In [88]:
df

,Manager_ID,Manager_Name,Team_Name,Team_ID
0,MAARS11,Mikel Arteta,Arsenal,ARS
1,FLARS16,Fredrik Ljungberg,Arsenal,ARS
2,AWARS12,Arsène Wenger,Arsenal,ARS
3,BRARS10,Bruce Rioch,Arsenal,ARS
4,SHARS14,Stewart Houston,Arsenal,ARS
...,...,...,...,...
273,GOWWA10,Gary O'Neil,Wolverhampton Wanderers,WWA
274,BMWWA28,Bruno Miguel Silva do Nascimento,Wolverhampton Wanderers,WWA
275,TCWWA11,Terry Connor,Wolverhampton Wanderers,WWA
276,MMWWA12,Mick McCarthy,Wolverhampton Wanderers,WWA


In [89]:
df.loc[df['Team_Name']=='Manchester United']

,Manager_ID,Manager_Name,Team_Name,Team_ID
145,EtMUN10,Erik ten Hag,Manchester United,MUN
146,RRMUN12,Ralf Rangnick,Manchester United,MUN
147,MCMUN14,Michael Carrick,Manchester United,MUN
148,LvMUN12,Louis van Gaal,Manchester United,MUN
149,RGMUN09,Ryan Giggs,Manchester United,MUN
150,OGMUN17,Ole Gunnar Solskjær,Manchester United,MUN
151,AFMUN12,Alex Ferguson,Manchester United,MUN


In [90]:
df.loc[df['Team_ID']=='SHU']

,Manager_ID,Manager_Name,Team_Name,Team_ID
190,PHSHU17,Paul Heckingbottom,Sheffield United,SHU
191,CWSHU11,Chris Wilder,Sheffield United,SHU


In [91]:
df.to_csv('Manager_Team_Table.csv')

In [92]:
df.loc[df['Team_Name']=='Barnsley']

,Manager_ID,Manager_Name,Team_Name,Team_ID


In [93]:
df.loc[df['Team_Name']=='Arsenal']

,Manager_ID,Manager_Name,Team_Name,Team_ID
0,MAARS11,Mikel Arteta,Arsenal,ARS
1,FLARS16,Fredrik Ljungberg,Arsenal,ARS
2,AWARS12,Arsène Wenger,Arsenal,ARS
3,BRARS10,Bruce Rioch,Arsenal,ARS
4,SHARS14,Stewart Houston,Arsenal,ARS


In [94]:
df.loc[df['Team_Name']=='Tottenham Hotspur']

,Manager_ID,Manager_Name,Team_Name,Team_ID
226,APTHO15,Ange Postecoglou,Tottenham Hotspur,THO
227,CSTHO16,Cristian Stellini,Tottenham Hotspur,THO
228,RMTHO09,Ryan Mason,Tottenham Hotspur,THO
229,ACTHO12,Antonio Conte,Tottenham Hotspur,THO
230,JRTHO11,Juande Ramos,Tottenham Hotspur,THO
231,CATHO10,Clive Allen,Tottenham Hotspur,THO
232,JMTHO31,José Mário dos Santos Mourinho Félix,Tottenham Hotspur,THO
233,JSTHO14,Jacques Santini,Tottenham Hotspur,THO
234,CGTHO14,Christian Gross,Tottenham Hotspur,THO
235,DPTHO10,David Pleat,Tottenham Hotspur,THO


In [95]:
df.loc[df['Team_Name']=='Nottingham Forest']

,Manager_ID,Manager_Name,Team_Name,Team_ID
173,NHNFO32,Nuno Herlander Simões Espírito Santo,Nottingham Forest,NFO
174,FCNFO10,Frank Clark,Nottingham Forest,NFO
175,BCNFO11,Brian Clough,Nottingham Forest,NFO
176,RANFO11,Ron Atkinson,Nottingham Forest,NFO


In [96]:
df.loc[df['Team_Name']=='Fulham']

,Manager_ID,Manager_Name,Team_Name,Team_ID
92,SJFUL16,Slavisa Jokanovic,Fulham,FUL
93,MAFUL28,Marco Alexandre Saraiva da Silva,Fulham,FUL
94,FMFUL11,Felix Magath,Fulham,FUL
95,RMFUL15,Rene Meulensteen,Fulham,FUL
96,RLFUL12,Ray Lewington,Fulham,FUL
97,LSFUL13,Lawrie Sanchez,Fulham,FUL
98,MJFUL09,Martin Jol,Fulham,FUL
99,CCFUL12,Chris Coleman,Fulham,FUL
100,JTFUL10,Jean Tigana,Fulham,FUL


In [97]:
df.loc[df['Team_Name']=='Wolverhampton Wanderers']

,Manager_ID,Manager_Name,Team_Name,Team_ID
272,SDWWA10,Steve Davis,Wolverhampton Wanderers,WWA
273,GOWWA10,Gary O'Neil,Wolverhampton Wanderers,WWA
274,BMWWA28,Bruno Miguel Silva do Nascimento,Wolverhampton Wanderers,WWA
275,TCWWA11,Terry Connor,Wolverhampton Wanderers,WWA
276,MMWWA12,Mick McCarthy,Wolverhampton Wanderers,WWA
277,DJWWA09,Dave Jones,Wolverhampton Wanderers,WWA


In [98]:
name_corrections = {
    'Nuno Herlander Simões Espírito Santo': 'Nuno Espírito Santo',
    'José Mário dos Santos Mourinho Félix': 'José Mourinho',
    'Marco Alexandre Saraiva da Silva': 'Marco Silva',
    'Bruno Miguel Silva do Nascimento': 'Bruno Lage'
}

df.loc[df['Manager_ID'] == 'BMWWA28','Manager_Name'] = name_corrections['Bruno Miguel Silva do Nascimento']
df.loc[df['Manager_ID'] == 'MAFUL28','Manager_Name'] = name_corrections['Marco Alexandre Saraiva da Silva']
df.loc[df['Manager_ID'] == 'JMTHO31','Manager_Name'] = name_corrections['José Mário dos Santos Mourinho Félix']
df.loc[df['Manager_ID'] == 'NHNFO32','Manager_Name'] = name_corrections['Nuno Herlander Simões Espírito Santo']

print(df[df['Manager_Name'].isin(name_corrections.keys())])

Empty DataFrame
Columns: [Manager_ID, Manager_Name, Team_Name, Team_ID]
Index: []


In [99]:
df.loc[df['Team_Name']=='Wolverhampton Wanderers']

,Manager_ID,Manager_Name,Team_Name,Team_ID
272,SDWWA10,Steve Davis,Wolverhampton Wanderers,WWA
273,GOWWA10,Gary O'Neil,Wolverhampton Wanderers,WWA
274,BMWWA28,Bruno Lage,Wolverhampton Wanderers,WWA
275,TCWWA11,Terry Connor,Wolverhampton Wanderers,WWA
276,MMWWA12,Mick McCarthy,Wolverhampton Wanderers,WWA
277,DJWWA09,Dave Jones,Wolverhampton Wanderers,WWA


In [100]:
df.loc[df['Team_Name']=='Fulham']

,Manager_ID,Manager_Name,Team_Name,Team_ID
92,SJFUL16,Slavisa Jokanovic,Fulham,FUL
93,MAFUL28,Marco Silva,Fulham,FUL
94,FMFUL11,Felix Magath,Fulham,FUL
95,RMFUL15,Rene Meulensteen,Fulham,FUL
96,RLFUL12,Ray Lewington,Fulham,FUL
97,LSFUL13,Lawrie Sanchez,Fulham,FUL
98,MJFUL09,Martin Jol,Fulham,FUL
99,CCFUL12,Chris Coleman,Fulham,FUL
100,JTFUL10,Jean Tigana,Fulham,FUL


In [101]:
df.loc[df['Team_Name']=='Tottenham Hotspur']

,Manager_ID,Manager_Name,Team_Name,Team_ID
226,APTHO15,Ange Postecoglou,Tottenham Hotspur,THO
227,CSTHO16,Cristian Stellini,Tottenham Hotspur,THO
228,RMTHO09,Ryan Mason,Tottenham Hotspur,THO
229,ACTHO12,Antonio Conte,Tottenham Hotspur,THO
230,JRTHO11,Juande Ramos,Tottenham Hotspur,THO
231,CATHO10,Clive Allen,Tottenham Hotspur,THO
232,JMTHO31,José Mourinho,Tottenham Hotspur,THO
233,JSTHO14,Jacques Santini,Tottenham Hotspur,THO
234,CGTHO14,Christian Gross,Tottenham Hotspur,THO
235,DPTHO10,David Pleat,Tottenham Hotspur,THO


In [102]:
df.loc[df['Team_Name']=='Nottingham Forest']

,Manager_ID,Manager_Name,Team_Name,Team_ID
173,NHNFO32,Nuno Espírito Santo,Nottingham Forest,NFO
174,FCNFO10,Frank Clark,Nottingham Forest,NFO
175,BCNFO11,Brian Clough,Nottingham Forest,NFO
176,RANFO11,Ron Atkinson,Nottingham Forest,NFO


In [103]:
df2 = pd.read_csv('EPL Manager History.csv')